Purpose - How accurately can draft position be predicted with just combine data and measureables
<br>
Problem - Can draft picks be predicted from combine and measureable data for incoming players?
<br>
Solution - Will create a classification(draft round) and regression model (draft pick)
<br>
Desired Outcome - Finding the limitations of the data source and seeing if past performance of the players is needed to improve accuracy.
<br>
Motivations - Grow data science skills while researching nfl stats. Will check against 2024 draft for accuracy.
<br>

https://www.datacamp.com/blog/a-beginner-s-guide-to-the-machine-learning-workflow
<br>
https://www.datacamp.com/blog/infographic-data-cleaning-checklist
<br>
https://towardsdatascience.com/workflow-of-a-machine-learning-project-ec1dba419b94

In [1]:
import pandas as pd
import nfl_data_py as nfl



In [2]:
years = range(2014,2024)

In [3]:
data = nfl.import_combine_data(years)

In [18]:
data.groupby('draft_year').count()

,season,draft_team,draft_round,draft_ovr,pfr_id,cfb_id,player_name,pos,school,ht,wt,forty,bench,vertical,broad_jump,cone,shuttle
draft_year,,,,,,,,,,,,,,,,,
2014.0,223,223,223,223,223,204,223,223,223,223,223,216,170,185,181,154,158
2015.0,213,213,213,213,213,199,213,213,213,213,213,212,155,173,171,149,155
2016.0,214,214,214,214,214,199,214,214,214,214,214,213,161,183,181,139,144
2017.0,222,222,222,222,222,211,222,222,222,222,222,189,167,189,188,143,144
2018.0,212,212,212,212,212,190,212,212,212,212,212,170,156,165,164,125,133
2019.0,217,217,217,217,217,208,217,217,217,217,217,178,164,177,175,135,143
2020.0,230,230,230,230,230,224,230,230,230,230,230,178,149,166,160,104,106
2021.0,193,193,193,193,193,189,193,193,193,178,179,167,141,170,158,137,141
2022.0,223,223,223,223,223,209,223,223,223,223,223,168,51,159,153,66,75


Data Cleaning
<br>
Dropping the columns of season, pfr_id, cfb_id,draft_year,draft_team
<br>
Dropping all rows with the target variables(draft_round,draft_ovr) missing. Hate the fact that it is a good chunk of the dataset
<br>

In [4]:
print(data.isnull().sum())

season            0
draft_year     1293
draft_team     1293
draft_round    1293
draft_ovr      1293
pfr_id          347
cfb_id          288
player_name       0
pos               0
school            0
ht               29
wt               24
forty           543
bench          1249
vertical        742
broad_jump      798
cone           1517
shuttle        1430
dtype: int64


In [6]:
#Dropping of columns
data.drop(['season', 'draft_year','draft_team','pfr_id','cfb_id'], axis=1,inplace=True)

In [7]:
print(data.isnull().sum())

draft_round    1293
draft_ovr      1293
player_name       0
pos               0
school            0
ht               29
wt               24
forty           543
bench          1249
vertical        742
broad_jump      798
cone           1517
shuttle        1430
dtype: int64


In [8]:
data = data.dropna(subset=['draft_round','draft_ovr'])

In [9]:
print(data.isnull().sum())

draft_round      0
draft_ovr        0
player_name      0
pos              0
school           0
ht              15
wt              14
forty          319
bench          761
vertical       429
broad_jump     467
cone           943
shuttle        890
dtype: int64


Next step is fill the missing values with the median of the value based on the position of the player
<br>
Code for filling with median of position works but the QB position has no bench press numbers. So to note this column may have to be dropped. 
<br>
Dropping K from the dataset
<br>


In [12]:
data[data['pos']=='QB'].isnull().sum()

draft_round      0
draft_ovr        0
player_name      0
pos              0
school           0
ht               0
wt               0
forty           25
bench          106
vertical        22
broad_jump      26
cone            34
shuttle         32
dtype: int64

In [13]:
data['pos'].unique()

array(['WR', 'S', 'CB', 'TE', 'OLB', 'ILB', 'OG', 'RB', 'QB', 'OT', 'DT',
       'DE', 'K', 'C', 'FB', 'P', 'LS', 'EDGE', 'LB', 'DB', 'OL', 'DL'],
      dtype=object)

In [17]:
for i in data.columns:
    print(i)

draft_round
draft_ovr
player_name
pos
school
ht
wt
forty
bench
vertical
broad_jump
cone
shuttle


In [18]:
data2 = data.copy(deep=True)

In [20]:
test = data2[data2['pos']== 'QB'] = data2[data2['pos']== 'QB'].fillna(data2[data2['pos']== 'QB'].median())

C:\Users\VonHinson\AppData\Local\Temp\ipykernel_16428\2975998413.py:1: FutureWarning: The default value of numeric_only in DataFrame.median is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  test = data2[data2['pos']== 'QB'] = data2[data2['pos']== 'QB'].fillna(data2[data2['pos']== 'QB'].median())


In [23]:
for pos in data2['pos'].unique():
    data2[data2['pos']== pos] = data2[data2['pos']== pos].fillna(data2[data2['pos']== pos].median())


C:\Users\VonHinson\AppData\Local\Temp\ipykernel_16428\1600264360.py:2: FutureWarning: The default value of numeric_only in DataFrame.median is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  data2[data2['pos']== pos] = data2[data2['pos']== pos].fillna(data2[data2['pos']== pos].median())
C:\Users\VonHinson\AppData\Local\Temp\ipykernel_16428\1600264360.py:2: FutureWarning: The default value of numeric_only in DataFrame.median is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  data2[data2['pos']== pos] = data2[data2['pos']== pos].fillna(data2[data2['pos']== pos].median())
C:\Users\VonHinson\AppData\Local\Temp\ipykernel_16428\1600264360.py:2: FutureWarning: The default value 

In [21]:
test.isnull().sum()

draft_round      0
draft_ovr        0
player_name      0
pos              0
school           0
ht               0
wt               0
forty            0
bench          106
vertical         0
broad_jump       0
cone             0
shuttle          0
dtype: int64

In [22]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2137 entries, 4569 to 7998
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   draft_round  2137 non-null   float64
 1   draft_ovr    2137 non-null   float64
 2   player_name  2137 non-null   object 
 3   pos          2137 non-null   object 
 4   school       2137 non-null   object 
 5   ht           2122 non-null   object 
 6   wt           2123 non-null   float64
 7   forty        1818 non-null   float64
 8   bench        1376 non-null   float64
 9   vertical     1708 non-null   float64
 10  broad_jump   1670 non-null   float64
 11  cone         1194 non-null   float64
 12  shuttle      1247 non-null   float64
dtypes: float64(9), object(4)
memory usage: 233.7+ KB


In [24]:
data2.isnull().sum()

draft_round      0
draft_ovr        0
player_name      0
pos              0
school           0
ht              15
wt               0
forty            0
bench          106
vertical        12
broad_jump      12
cone            12
shuttle         12
dtype: int64

In [27]:
for pos in data2['pos'].unique():
    print(pos)
    print(data2[data2['pos']== pos].isnull().sum())

WR
draft_round    0
draft_ovr      0
player_name    0
pos            0
school         0
ht             2
wt             0
forty          0
bench          0
vertical       0
broad_jump     0
cone           0
shuttle        0
dtype: int64
S
draft_round    0
draft_ovr      0
player_name    0
pos            0
school         0
ht             4
wt             0
forty          0
bench          0
vertical       0
broad_jump     0
cone           0
shuttle        0
dtype: int64
CB
draft_round    0
draft_ovr      0
player_name    0
pos            0
school         0
ht             0
wt             0
forty          0
bench          0
vertical       0
broad_jump     0
cone           0
shuttle        0
dtype: int64
TE
draft_round    0
draft_ovr      0
player_name    0
pos            0
school         0
ht             0
wt             0
forty          0
bench          0
vertical       0
broad_jump     0
cone           0
shuttle        0
dtype: int64
OLB
draft_round    0
draft_ovr      0
player_name    

In [28]:
data2 = data2[data2.pos != 'K']

In [29]:
for pos in data2['pos'].unique():
    print(pos)
    print(data2[data2['pos']== pos].isnull().sum())

WR
draft_round    0
draft_ovr      0
player_name    0
pos            0
school         0
ht             2
wt             0
forty          0
bench          0
vertical       0
broad_jump     0
cone           0
shuttle        0
dtype: int64
S
draft_round    0
draft_ovr      0
player_name    0
pos            0
school         0
ht             4
wt             0
forty          0
bench          0
vertical       0
broad_jump     0
cone           0
shuttle        0
dtype: int64
CB
draft_round    0
draft_ovr      0
player_name    0
pos            0
school         0
ht             0
wt             0
forty          0
bench          0
vertical       0
broad_jump     0
cone           0
shuttle        0
dtype: int64
TE
draft_round    0
draft_ovr      0
player_name    0
pos            0
school         0
ht             0
wt             0
forty          0
bench          0
vertical       0
broad_jump     0
cone           0
shuttle        0
dtype: int64
OLB
draft_round    0
draft_ovr      0
player_name    

In [30]:
data2.isnull().sum()

draft_round      0
draft_ovr        0
player_name      0
pos              0
school           0
ht              15
wt               0
forty            0
bench          106
vertical         0
broad_jump       0
cone             0
shuttle          0
dtype: int64

In [31]:
data2.bench.fillna(0, inplace=True)

In [32]:
data2.isnull().sum()

draft_round     0
draft_ovr       0
player_name     0
pos             0
school          0
ht             15
wt              0
forty           0
bench           0
vertical        0
broad_jump      0
cone            0
shuttle         0
dtype: int64

In [33]:
data2 = data2[data2['ht'].notna()]

In [34]:
data2.isnull().sum()

draft_round    0
draft_ovr      0
player_name    0
pos            0
school         0
ht             0
wt             0
forty          0
bench          0
vertical       0
broad_jump     0
cone           0
shuttle        0
dtype: int64

In [35]:
data2.describe()

,draft_round,draft_ovr,wt,forty,bench,vertical,broad_jump,cone,shuttle
count,2110.000000,2110.000000,2110.000000,2110.000000,2110.000000,2110.000000,2110.000000,2110.000000,2110.000000
mean,3.790521,115.383412,244.009479,4.732116,19.337678,33.309147,117.323460,7.221645,4.376036
std,1.891486,70.734877,46.039904,0.292454,7.055264,3.993602,8.799448,0.357321,0.224774
min,1.000000,1.000000,155.000000,4.220000,0.000000,19.500000,82.000000,6.450000,3.810000
25%,2.000000,55.000000,205.000000,4.500000,15.000000,30.500000,112.000000,6.960000,4.210000
50%,4.000000,110.000000,234.000000,4.640000,20.000000,33.500000,118.500000,7.090000,4.310000
75%,5.000000,171.000000,286.750000,4.920000,24.000000,36.000000,123.000000,7.450000,4.500000
max,7.000000,262.000000,384.000000,5.850000,44.000000,45.000000,147.000000,8.410000,5.380000


In [36]:
data2.to_pickle('draft_combine_2014_2023_NoNan.pkl')

Need to drop the player name column
<br>
Categoric the school and pos columns. Change the height column to a number(total inches)

In [37]:
data2.nunique()

draft_round       7
draft_ovr       258
player_name    2104
pos              21
school          224
ht               17
wt              188
forty           134
bench            42
vertical         53
broad_jump       58
cone            182
shuttle         126
dtype: int64

In [39]:
data2.school.value_counts()

Alabama          84
LSU              66
Georgia          65
Florida          57
Ohio State       53
                 ..
Villanova         1
Akron             1
West Georgia      1
Georgia State     1
Ball St.          1
Name: school, Length: 224, dtype: int64